In [1]:
import sys
sys.path.append('/home/sfang/windows/gitlab/stanleysfang/surveillance_2019_ncov/prod')

In [2]:
from google.cloud import bigquery
from BigQueryWrapper import QueryRunner, Loader, Extractor
import pandas as pd
import datetime
import re

In [3]:
project_id = 'stanleysfang'

In [4]:
client = bigquery.Client(project=project_id)

In [5]:
qr = QueryRunner(client=client)
loader = Loader(client=client)
extractor = Extractor(client=client)

In [6]:
geo='us'

### Functions

In [7]:
def print_job_result(job, client, max_results=20):
    job.result()
    bq_table = client.get_table(job.destination)
    df = client.list_rows(bq_table, max_results=max_results).to_dataframe()
    print(bq_table.full_table_id)
    print(df.head(max_results))

### Load Data

In [8]:
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')

In [9]:
confirmed.iloc[:, 0:15]

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,0,0,0
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",0,0,0,0
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",0,0,0,0
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",0,0,0,0
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,"Teton, Wyoming, US",0,0,0,0
3336,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,"Uinta, Wyoming, US",0,0,0,0
3337,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,"Unassigned, Wyoming, US",0,0,0,0
3338,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,"Washakie, Wyoming, US",0,0,0,0


In [10]:
deaths.iloc[:, 0:15]

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population,1/22/20,1/23/20,1/24/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",55869,0,0,0
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",223234,0,0,0
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",24686,0,0,0
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",22394,0,0,0
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",57826,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,"Teton, Wyoming, US",23464,0,0,0
3336,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,"Uinta, Wyoming, US",20226,0,0,0
3337,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,"Unassigned, Wyoming, US",0,0,0,0
3338,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,"Washakie, Wyoming, US",7805,0,0,0


In [11]:
dt_cols = [datetime.datetime.strptime(dt, '%m/%d/%y').strftime('dt_%Y%m%d') for dt in list(confirmed.columns)[11:]]

#### confirmed

In [12]:
confirmed_cols = ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'county', 'province_state', 'country_region', 'latitude', 'longitude', 'combined_key'] + dt_cols
confirmed.columns = confirmed_cols

In [13]:
confirmed_schema = [
    ('UID', 'INT64'),
    ('iso2', 'STRING'),
    ('iso3', 'STRING'),
    ('code3', 'INT64'),
    ('FIPS', 'FLOAT64'),
    ('county', 'STRING'),
    ('province_state', 'STRING'),
    ('country_region', 'STRING'),
    ('latitude', 'FLOAT64'),
    ('longitude', 'FLOAT64'),
    ('combined_key', 'STRING'),
] + [(dt_col, 'INT64') for dt_col in dt_cols]

In [14]:
loader.load_df(confirmed, 'stanleysfang.surveillance_2019_ncov.ts_2019_ncov_{geo}_confirmed_raw'.format(geo=geo), schema=confirmed_schema)

#### deaths

In [15]:
deaths_cols = confirmed_cols
deaths_cols.insert(11, 'population')
deaths.columns = deaths_cols

In [16]:
deaths_schema = confirmed_schema
deaths_schema.insert(11, ('population', 'INT64'))

In [17]:
loader.load_df(deaths, 'stanleysfang.surveillance_2019_ncov.ts_2019_ncov_{geo}_deaths_raw'.format(geo=geo), schema=deaths_schema)

In [18]:
for job in loader.job_history:
    print_job_result(job, client)

/home/sfang/anaconda3/envs/surveillance_2019_ncov/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  after removing the cwd from sys.path.


stanleysfang:surveillance_2019_ncov.ts_2019_ncov_us_confirmed_raw
         UID iso2 iso3  code3     FIPS          county  province_state  \
0   84080001   US  USA    840  80001.0       Out of AL         Alabama   
1   84090001   US  USA    840  90001.0      Unassigned         Alabama   
2   84002013   US  USA    840   2013.0  Aleutians East          Alaska   
3   84002016   US  USA    840   2016.0  Aleutians West          Alaska   
4   84002060   US  USA    840   2060.0     Bristol Bay          Alaska   
5   84002068   US  USA    840   2068.0          Denali          Alaska   
6   84002100   US  USA    840   2100.0          Haines          Alaska   
7   84002105   US  USA    840   2105.0   Hoonah-Angoon          Alaska   
8   84002158   US  USA    840   2158.0        Kusilvak          Alaska   
9   84080002   US  USA    840  80002.0       Out of AK          Alaska   
10  84002230   US  USA    840   2230.0         Skagway          Alaska   
11  84090002   US  USA    840  90002.0      Un

/home/sfang/anaconda3/envs/surveillance_2019_ncov/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  after removing the cwd from sys.path.


stanleysfang:surveillance_2019_ncov.ts_2019_ncov_us_deaths_raw
         UID iso2 iso3  code3     FIPS        county  province_state  \
0   84001045   US  USA    840   1045.0          Dale         Alabama   
1   84013305   US  USA    840  13305.0         Wayne         Georgia   
2   84090017   US  USA    840  90017.0    Unassigned        Illinois   
3   84080036   US  USA    840  80036.0     Out of NY        New York   
4   84037087   US  USA    840  37087.0       Haywood  North Carolina   
5   84038015   US  USA    840  38015.0      Burleigh    North Dakota   
6   84047179   US  USA    840  47179.0    Washington       Tennessee   
7   84048025   US  USA    840  48025.0           Bee           Texas   
8   84048249   US  USA    840  48249.0     Jim Wells           Texas   
9   84048409   US  USA    840  48409.0  San Patricio           Texas   
10  84048465   US  USA    840  48465.0     Val Verde           Texas   
11  84005051   US  USA    840   5051.0       Garland        Arkansas   
1

### Dataprep

#### restructure raw data

In [19]:
for metric in ['confirmed', 'deaths']:
    array_query = ''
    for dt_col in dt_cols:
        array_query = \
        """{array_query}
        STRUCT(PARSE_DATE('dt_%Y%m%d', '{dt_col}') AS dt, {dt_col} AS total_{metric}),
        """.format(array_query=array_query, dt_col=dt_col, metric=metric)
    array_query = re.sub('[,\n ]*$', '\n', array_query)
    
    query = \
    """
    SELECT dt, a.* EXCEPT(arr), total_{metric}
    FROM (
        SELECT
            county, province_state, country_region, combined_key, latitude, longitude,
            [{array_query}] AS arr
        FROM `stanleysfang.surveillance_2019_ncov.ts_2019_ncov_{geo}_{metric}_raw`
    ) a, UNNEST(arr)
    """.format(metric=metric, array_query=array_query, geo=geo)
    
    query_job = qr.run_query(query, destination_table='stanleysfang.surveillance_2019_ncov.ts_2019_ncov_{geo}_{metric}'.format(geo=geo, metric=metric), time_partitioning=True, partition_field='dt')

In [20]:
for job in qr.job_history:
    print_job_result(job, client)

/home/sfang/anaconda3/envs/surveillance_2019_ncov/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  after removing the cwd from sys.path.


stanleysfang:surveillance_2019_ncov.ts_2019_ncov_us_confirmed
            dt          county  province_state country_region  \
0   2020-02-12       Out of AL         Alabama             US   
1   2020-02-12      Unassigned         Alabama             US   
2   2020-02-12  Aleutians East          Alaska             US   
3   2020-02-12  Aleutians West          Alaska             US   
4   2020-02-12     Bristol Bay          Alaska             US   
5   2020-02-12          Denali          Alaska             US   
6   2020-02-12          Haines          Alaska             US   
7   2020-02-12   Hoonah-Angoon          Alaska             US   
8   2020-02-12        Kusilvak          Alaska             US   
9   2020-02-12       Out of AK          Alaska             US   
10  2020-02-12         Skagway          Alaska             US   
11  2020-02-12      Unassigned          Alaska             US   
12  2020-02-12        Wrangell          Alaska             US   
13  2020-02-12         Yakut

/home/sfang/anaconda3/envs/surveillance_2019_ncov/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  after removing the cwd from sys.path.


stanleysfang:surveillance_2019_ncov.ts_2019_ncov_us_deaths
            dt        county  province_state country_region  \
0   2020-02-05          Dale         Alabama             US   
1   2020-02-05         Wayne         Georgia             US   
2   2020-02-05    Unassigned        Illinois             US   
3   2020-02-05     Out of NY        New York             US   
4   2020-02-05       Haywood  North Carolina             US   
5   2020-02-05      Burleigh    North Dakota             US   
6   2020-02-05    Washington       Tennessee             US   
7   2020-02-05           Bee           Texas             US   
8   2020-02-05     Jim Wells           Texas             US   
9   2020-02-05  San Patricio           Texas             US   
10  2020-02-05     Val Verde           Texas             US   
11  2020-02-05       Garland        Arkansas             US   
12  2020-02-05       Laurens         Georgia             US   
13  2020-02-05      Kootenai           Idaho             US

#### agg data

In [21]:
ts_2019_ncov_query = \
"""
SELECT
    dt, county, province_state, country_region, combined_key, latitude, longitude,
    population,
    total_confirmed, total_deaths, daily_new_confirmed, daily_new_deaths,
    ROUND(AVG(daily_new_confirmed) OVER(PARTITION BY combined_key ORDER BY dt ROWS BETWEEN 6 PRECEDING AND CURRENT ROW), 1) AS daily_new_confirmed_7d_ma,
    ROUND(AVG(daily_new_deaths) OVER(PARTITION BY combined_key ORDER BY dt ROWS BETWEEN 6 PRECEDING AND CURRENT ROW), 1) AS daily_new_deaths_7d_ma,
    ROUND(AVG(daily_new_confirmed) OVER(PARTITION BY combined_key ORDER BY dt ROWS BETWEEN 27 PRECEDING AND CURRENT ROW), 1) AS daily_new_confirmed_28d_ma,
    ROUND(AVG(daily_new_deaths) OVER(PARTITION BY combined_key ORDER BY dt ROWS BETWEEN 27 PRECEDING AND CURRENT ROW), 1) AS daily_new_deaths_28d_ma,
    IF(population = 0, NULL, ROUND(total_confirmed/population, 4)) AS incident_rate,
    IF(total_confirmed = 0, NULL, ROUND(total_deaths/total_confirmed, 4)) AS case_fatality_rate,
    MAX(dt) OVER() AS last_update_dt,
    TIMESTAMP(REGEXP_REPLACE(STRING(CURRENT_TIMESTAMP, "America/Los_Angeles"), r'[\+-][0-9]{{2}}$', '')) AS last_updated_ts -- need the double bracket to avoid error with str.format
FROM (
    SELECT
        * EXCEPT(daily_new_confirmed, daily_new_deaths),
        IFNULL(daily_new_confirmed, total_confirmed) AS daily_new_confirmed,
        IFNULL(daily_new_deaths, total_deaths) AS daily_new_deaths
    FROM (
        SELECT
            *,
            total_confirmed - LAG(total_confirmed) OVER(PARTITION BY combined_key ORDER BY dt) AS daily_new_confirmed,
            total_deaths - LAG(total_deaths) OVER(PARTITION BY combined_key ORDER BY dt) AS daily_new_deaths
        FROM (
            SELECT
                a.*, b.population
            FROM (
                SELECT
                    COALESCE(a.dt, b.dt) AS dt,
                    COALESCE(a.county, b.county) AS county,
                    COALESCE(a.province_state, b.province_state) AS province_state,
                    COALESCE(a.country_region, b.country_region) AS country_region,
                    COALESCE(a.combined_key, b.combined_key) AS combined_key,
                    COALESCE(a.latitude, b.latitude) AS latitude,
                    COALESCE(a.longitude, b.longitude) AS longitude,
                    IFNULL(a.total_confirmed, 0) AS total_confirmed,
                    IFNULL(b.total_deaths, 0) AS total_deaths
                FROM `stanleysfang.surveillance_2019_ncov.ts_2019_ncov_{geo}_confirmed` a
                FULL JOIN `stanleysfang.surveillance_2019_ncov.ts_2019_ncov_{geo}_deaths` b
                ON a.dt = b.dt AND a.combined_key = b.combined_key
            ) a
            LEFT JOIN `stanleysfang.surveillance_2019_ncov.ts_2019_ncov_{geo}_deaths_raw` b
            ON a.combined_key = b.combined_key
        )
    )
)
""".format(geo=geo)

In [22]:
query_job = qr.run_query(ts_2019_ncov_query, destination_table='stanleysfang.surveillance_2019_ncov.ts_2019_ncov_{geo}'.format(geo=geo), time_partitioning=True, partition_field='dt')

In [23]:
print_job_result(query_job, client)

/home/sfang/anaconda3/envs/surveillance_2019_ncov/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  after removing the cwd from sys.path.


stanleysfang:surveillance_2019_ncov.ts_2019_ncov_us
            dt      county  province_state country_region  \
0   2020-01-27        Bell        Kentucky             US   
1   2020-01-27     Emanuel         Georgia             US   
2   2020-01-27      Gordon         Georgia             US   
3   2020-01-27     Jackson        Arkansas             US   
4   2020-01-27     Madison        Arkansas             US   
5   2020-01-27   Nez Perce           Idaho             US   
6   2020-01-27      Pierce        Nebraska             US   
7   2020-01-27   Pittsburg        Oklahoma             US   
8   2020-01-27        Polk            Iowa             US   
9   2020-01-27        Rice          Kansas             US   
10  2020-01-27  Rio Grande     Puerto Rico             US   
11  2020-01-27      Sumner          Kansas             US   
12  2020-01-27   Trousdale       Tennessee             US   
13  2020-01-27   Wilkinson     Mississippi             US   
14  2020-01-27    Fillmore       

#### US current table

In [24]:
us_cur_query = \
"""
SELECT *
FROM `stanleysfang.surveillance_2019_ncov.ts_2019_ncov_{geo}`
WHERE dt = last_update_dt
""".format(geo=geo)

In [25]:
query_job = qr.run_query(us_cur_query, destination_table='stanleysfang.surveillance_2019_ncov.ts_2019_ncov_{geo}_cur'.format(geo=geo))

In [26]:
print_job_result(query_job, client)

/home/sfang/anaconda3/envs/surveillance_2019_ncov/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  after removing the cwd from sys.path.


stanleysfang:surveillance_2019_ncov.ts_2019_ncov_us_cur
            dt      county            province_state country_region  \
0   2020-11-13   Out of RI              Rhode Island             US   
1   2020-11-13  Unassigned             New Hampshire             US   
2   2020-11-13  Unassigned               Connecticut             US   
3   2020-11-13   Out of NH             New Hampshire             US   
4   2020-11-13        None          Diamond Princess             US   
5   2020-11-13  Unassigned                    Hawaii             US   
6   2020-11-13       Kauai                    Hawaii             US   
7   2020-11-13   Out of DC      District of Columbia             US   
8   2020-11-13        None            American Samoa             US   
9   2020-11-13   Out of CT               Connecticut             US   
10  2020-11-13   Out of DE                  Delaware             US   
11  2020-11-13  Unassigned                  Delaware             US   
12  2020-11-13     Ka

### Extract Table

In [27]:
extract_job = extractor.extract(query_job.destination, 'gs://surveillance_2019_ncov/ts_2019_ncov_{geo}_cur.csv'.format(geo=geo))

In [28]:
extract_job.result()